In [21]:
import pickle
import pandas as pd
import sklearn

In [22]:
year = 2024
month = 1
taxi_type = 'green'

input_file = f'/workspaces/mle-zoomcamp-prj/project/src_data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'

output_file = f'output/{taxi_type}/{year:04d}-{month:02d}.parquet'

with open('/workspaces/mle-zoomcamp-prj/project/models/lin_reg.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)
    

In [23]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [24]:
        df = read_data(input_file)

In [25]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [26]:
y_pred.std()

5.329070518200751e-15

In [27]:
!mkdir output

mkdir: cannot create directory ‘output’: File exists


In [28]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [29]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

In [30]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [31]:
!ls -lh output

total 4.0K
drwxrwxrwx+ 2 codespace codespace 4.0K Jul  4 06:43 green
